# EMIS and CapitalIQ person network
We want to build a person network based on company information. Those persons include board members, CEOs, directors, and shareholders. We connect two persons if they are associated with the same company. Here, we first focus on EMIS dataset. After that, we look into CapitalIQ data. Finally, we combine data from EMIS and CapitalIQ.

## EMIS key executives and shareholders network

### Build dictionary for neighbors

In [1]:
import pandas as pd
df_comp_info = pd.read_pickle('../../Database/EMIS/preprocessed/comp_info_human_20180310.pickle')
print(df_comp_info.shape)

(14831, 30)


In [2]:
df_comp_info.iloc[:5,:16]

,Index_Num,Company,Key_Executives,Employee_Number,Employee_Info_Year,Financial_Auditors,Main_Products,Phone,Email,Business_Description,Shareholder_String,Revenue_2017,Revenue_2016,Revenue_2015,Revenue_2014,Revenue_2013
0,1.0,Petroleo Brasileiro S.A.,Murilo Pinto de Oliveira Ferreira (Chairman of...,68829.0,2016.0,PwC,"petroleum, automotive gasoline, diesel, natura...",+55 21 3224-2040,acionistas@petrobras.com.br,The company Petrobras was founded in October 1...,"União Federal (28.67%), Bndes Participacoes S....",NaN,80952.74,NaN,NaN,NaN
1,1.0,Petroleo Brasileiro S.A.,Murilo Pinto de Oliveira Ferreira (Chairman of...,68829.0,2016.0,PwC,"petroleum, automotive gasoline, diesel, natura...",+55 21 3224-2040,acionistas@petrobras.com.br,The company Petrobras was founded in October 1...,"União Federal (28.67%), Bndes Participacoes S....",NaN,80952.74,NaN,NaN,NaN
2,1.0,Petroleo Brasileiro S.A.,Murilo Pinto de Oliveira Ferreira (Chairman of...,68829.0,2016.0,PwC,"petroleum, automotive gasoline, diesel, natura...",+55 21 3224-2040,acionistas@petrobras.com.br,The company Petrobras was founded in October 1...,"União Federal (28.67%), Bndes Participacoes S....",NaN,80952.74,NaN,NaN,NaN
3,2.0,Banco Santander S.A.,Ana Patricia Botin-Sanz de Sautuola Y o'Shea (...,NaN,NaN,Deloitte,NaN,+55 34 90211-2211,acionistasbdr@santander.com.br,"Banco Santander S.A. was founded in 1857, head...","Blackrock, INC (5.01%)",59816.49,NaN,NaN,NaN,NaN
4,3.0,Jbs S.A.,Joesley Mendonca Batista (Chairman of the Boar...,225422.0,2016.0,BDO,beef manufacturing,"+55 11 3144-4447, +55 11 3144-4224",ri@jbs.com.br,"Initiating as a meat trader in 1953, currently...","Fb Participacoes S.A. (42.31%), Bndes Particip...",NaN,48977.85,NaN,NaN,NaN


In [3]:
df_comp_info.iloc[:5,16:]

,Revenue_2012,Revenue_Estimated_Klooks,Industry_List,NAICS_Num_List,Status,CNPJ,Audited,Consolidated,Shareholder_Num,Shareholder_Index,Shareholder,Equity,Shareholder_Name_Normalized,Shareholder_Is_Human
0,NaN,NaN,Petroleum Refineries ; Natural Gas Extraction ...,32411;21113;21112,0,33000167000101,1,1,3,0,União Federal,28.67,federal uniao,False
1,NaN,NaN,Petroleum Refineries ; Natural Gas Extraction ...,32411;21113;21112,0,33000167000101,1,1,3,1,Bndes Participacoes S.A. - Bndespar,9.67,bndes bndespar participacoes s.a.,False
2,NaN,NaN,Petroleum Refineries ; Natural Gas Extraction ...,32411;21113;21112,0,33000167000101,1,1,3,2,Banco Nacional De Desenvolvimento Economico E ...,6.87,banco de desenvolvimento e economico nacional ...,False
3,NaN,NaN,"Securities, Commodity Contracts, and Other Fin...",523;522,0,05707616000110,1,1,1,0,Blackrock INC,5.01,blackrock inc,False
4,NaN,NaN,Manufacturing ; Animal Slaughtering and Proc...,31151;3116;316,0,02916265000160,1,1,2,0,Fb Participacoes S.A.,42.31,fb participacoes s.a.,False


In [4]:
# Add shareholders into dictionary for persons
from collections import defaultdict
dict_person_id = dict()
dict_id_person = dict()
dict_id_person_origin = dict()
dict_id_neighbor = defaultdict(set)
dict_id_comp_info = defaultdict(list) # The list includes a set of 'Company_Profile' and a set of 'Company_CNPJ'

# Collect human shareholder names
p_cnt = 0
for idx, row in df_comp_info.iterrows():
    if row["Shareholder_Is_Human"] and not dict_person_id.has_key(row["Shareholder_Name_Normalized"]):
        dict_person_id[row["Shareholder_Name_Normalized"]] = p_cnt
        dict_id_person[p_cnt] = row["Shareholder_Name_Normalized"]
        dict_id_person_origin[p_cnt] = row["Shareholder"]
        dict_id_neighbor[p_cnt] = set()
        dict_id_comp_info[p_cnt] = [set(), set()]
        dict_id_comp_info[p_cnt][0].add(row['Company'])
        dict_id_comp_info[p_cnt][1].add(row['CNPJ'])
        p_cnt += 1
    elif dict_person_id.has_key(row["Shareholder_Name_Normalized"]):
        dict_id_comp_info[dict_person_id[row["Shareholder_Name_Normalized"]]][0].add(row['Company'])
        dict_id_comp_info[dict_person_id[row["Shareholder_Name_Normalized"]]][1].add(row['CNPJ'])
        
# Fill in neighbors
for idx, row in df_comp_info.iterrows():
    if row['Shareholder_Is_Human']:
        df_comp_info_same_shareholder_temp = df_comp_info[df_comp_info['CNPJ']==row['CNPJ']]
        for idx1, row1 in df_comp_info_same_shareholder_temp.iterrows():
            if row1['Shareholder_Is_Human'] and (row['Shareholder_Name_Normalized'] != row1['Shareholder_Name_Normalized']):
                dict_id_neighbor[dict_person_id[row['Shareholder_Name_Normalized']]].add(dict_person_id[row1['Shareholder_Name_Normalized']])

In [5]:
dict_id_neighbor

defaultdict(set,
            {0: {495, 1225, 1661, 1662, 1812, 1842, 2427},
             1: set(),
             2: set(),
             3: set(),
             4: {5},
             5: {4},
             6: {7, 8, 9, 10},
             7: {6, 8, 9, 10},
             8: {6, 7, 9, 10},
             9: {6, 7, 8, 10},
             10: {6, 7, 8, 9},
             11: set(),
             12: set(),
             13: {14},
             14: {13},
             15: {16},
             16: {15},
             17: {18, 19, 20, 21, 22, 23},
             18: {17, 19, 20, 21, 22, 23},
             19: {17, 18, 20, 21, 22, 23},
             20: {17, 18, 19, 21, 22, 23},
             21: {17, 18, 19, 20, 22},
             22: {17, 18, 19, 20, 21},
             23: {17, 18, 19, 20},
             24: set(),
             25: set(),
             26: set(),
             27: {28, 29, 1402},
             28: {27, 29},
             29: {27, 28},
             30: set(),
             31: {32, 33},
             32: {31, 3

In [19]:
with open('shareholder_network.csv','w') as myfile:
    for node, neighs in dict_id_neighbor.iteritems():
        line_temp = []
        line_temp.append(str(node))
        neighs_ls = list(neighs)
        line_temp.extend([str(neigh) for neigh in neighs_ls])
        myfile.write(','.join(line_temp)+'\n')

In [6]:
df_HNWI_10 = pd.read_excel('../../Database/EMIS/preprocessed/prediction_20180322/HNWI_10_ranked.xlsx')
df_HNWI_10[:5]

,Shareholder,Company_Profile,Company_CNPJ,Total_Liquidity,Main_Products,Business_Description,Phone,Email,City,State_County,Address
151,Roberto Marinho,Globo Comunicacao e Participacoes S.A.,27865757000102,9077.729062,NaN,Globo Comicações e Participacoes is a holding ...,+55 (21) 2540-2623,NaN,Rio de Janeiro (City),Rio de Janeiro (Region),"R. Lopes Quintas, 303"
449,Marcelo Henrique Ribeiro Alecrim,Sat Participacoes S.A.,05749030000119,5617.167786,NaN,The company Sat Participacoes was founded in 2...,+55 (84) 4006-7878 / (84) 4006-7889,joao.jaome@grupocarau.com.br\t,Parnamirim,Rio Grande do Norte,"R Jose Peixoto 283, Sala 02 Andar 1"
1036,Joseph Yacoub Safra,Banco Safra S.A.;Serasa S.A.,58160789000128;62173620000180,2996.340358,nan;nan,Banco Safra SA has as its corporate purpose th...,+55 (11) 3175-7575;+55-19-33733797; +55-11-400...,atendimento.regulamentar@safra.com.br;amadorro...,Sao Paulo (City);Sao Paulo (City),Sao Paulo (Region);Sao Paulo (Region),"Avenida Paulista, 2100, Cerqueira Cesar;Al Dos..."
522,Adalberto Graf,Pro Vale Securitizadora de Ativos Empresariais...,10385214000187,2928.040878,NaN,The company Pro Vale Securitizadora de Ativos ...,NaN,juliad@invepar.com.br,Indaial,Santa Catarina,R Lauro Muller 18 S 304 - Centro
740,Bb Carteira Ativa,Litel Participacoes S.A.,00743065000127,1989.952641,NaN,Litel Participacoes is a publicly traded corpo...,NaN,governance@bnymellon.com.br,Rio de Janeiro (City),Rio de Janeiro (Region),"Av. Presidente Wilson, 231 - 11 Andar - Centro"


In [25]:
import re
# The pattern for non-alphanumeric characters are coming from Jupyter Notebook ExperimentRegression
patt_non_alphanum = r'[\.\&/:,\'\"\-;\\+=\#`!?>\|\_]' 
# Note that have to add escape sign before '-', o.w. it is interpreted as a range symbol and as a result, numbers are ignored

def Normalized_Name_Str(name, patt_non_alphanum, include_parenth=False):
    """Normalize a string as a name."""
    comp_name = unidecode(name.strip().lower())
    normalized_list = []
    para_ele = re.findall(r'[\(\[](.+)[\)\]]',comp_name)# Extract inside () or []
    if para_ele:
        comp_name = re.sub(r'\(.+\)', ' ', comp_name)
        if include_parenth:
            para_ele = [y for x in para_ele for y in re.sub(patt_non_alphanum,' ', x).split()]# Flatten a list of lists
            normalized_list.extend(para_ele)
    sa_ele = re.findall(r' s[\./]{1}a[\.]?',comp_name)# Extract "s.a." or "s/a", and put "s.a." back in the string.
    if sa_ele:
        for _ in range(len(sa_ele)):
            normalized_list.append('s.a.')
        comp_name = re.sub(r's[\./]{1}a[\.]?', ' ', comp_name)
    ltd_ele = re.findall(r' ltd[a]?[.]?',comp_name)# Extract "ltda." or "ltda" or "ltd"; and put "ltd" back in the string.
    if ltd_ele:
        for _ in range(len(ltd_ele)):
            normalized_list.append('ltd')
        comp_name = re.sub(r'ltd[a]?[.]?', ' ', comp_name)
    lp_ele = re.findall(r' l[\./]{1}p[\.]?',comp_name)# Extract "l.p." or "l.p" or "lp"; and put "lp" back in the string.
    if lp_ele:
        for _ in range(len(lp_ele)):
            normalized_list.append('lp')
        comp_name = re.sub(r'l[\./]{1}p[\.]?', ' ', comp_name)
    sl_ele = re.findall(r' s[\./]{1}l[\.]?',comp_name)# Extract "s.l." or "s.l" or "sl"; and put "sl" back in the string.
    if sl_ele:
        for _ in range(len(sl_ele)):
            normalized_list.append('sl')
        comp_name = re.sub(r's[\./]{1}l[\.]?', ' ', comp_name)
    cv_ele = re.findall(r' c[\./]{1}v[\.]?',comp_name)# Extract "c.v." or "c.v" or "cv"; and put "cv" back in the string.
    if cv_ele:
        for _ in range(len(cv_ele)):
            normalized_list.append('cv')
        comp_name = re.sub(r'c[\./]{1}v[\.]?', ' ', comp_name)
    bv_ele = re.findall(r' b[\./]{1}v[\.]?',comp_name)# Extract "b.v." or "b.v" or "bv"; and put "bv" back in the string.
    if bv_ele:
        for _ in range(len(bv_ele)):
            normalized_list.append('bv')
        comp_name = re.sub(r'b[\./]{1}v[\.]?', ' ', comp_name)

    comp_name = re.sub(patt_non_alphanum,' ', comp_name)
    normalized_list.extend(comp_name.split())
#     print idx, normalized_list, comp_name.split()
    normalized_list.sort()
    return ' '.join(normalized_list)

In [31]:
from unidecode import unidecode
with open('../../Database/EMIS/preprocessed/HWNI_10_shareholder_network_origin_name.csv','w') as myfile:
    for idx, row in df_HNWI_10.iterrows():
        sh_name = row["Shareholder"]
        sh_name_normalized = Normalized_Name_Str(sh_name, patt_non_alphanum)
        if sh_name_normalized in dict_person_id:
            line_temp = []
            line_temp.append(str(idx))
            line_temp.append(re.sub(r',',' ',unidecode(dict_id_person_origin[dict_person_id[sh_name_normalized]])))
            neighs_id_ls = list(dict_id_neighbor[dict_person_id[sh_name_normalized]])
            line_temp.extend([re.sub(r',',' ',unidecode(dict_id_person_origin[neigh_id])) for neigh_id in neighs_id_ls])
            myfile.write(','.join(line_temp)+'\n')
        else:
            print("!!!There is no {} in the dict.".format(row["Shareholder"],))

In [32]:
df_HNWI_1 = pd.read_excel('../../Database/EMIS/preprocessed/prediction_20180322/HNWI_1_ranked.xlsx')
df_HNWI_1[:5]

,Shareholder,Company_Profile,Company_CNPJ,Total_Liquidity,Main_Products,Business_Description,Phone,Email,City,State_County,Address
1004,Rolim Adolfo Amaro,Tam Aviacao Executiva E Taxi Aereo S.A.,52045457000116,9.985099,"Aircraft Trade, Chartering and Management.",TAM Aviação Executiva is a business division o...,+55-11-55828836; +55-11-50313988; +55-11-20890...,NaN,Sao Paulo (City),Sao Paulo (Region),R Mons Antonio Pepe 94 Aeroporto
1186,Salete Teresinha Gorgen,Risa S.A.,06855894000188,9.898843,NaN,"The company Risa was founded in 1982, headquar...",+55-89-3544-3055 ; +55-89-3570-1152,contato@risasa.com,Baixa Grande do Ribeiro,Piaui,"Lug Fazenda Ribeirao, Zona Rural"
276,Eunice Goncalves Almeida,Goncalves S A Industria Grafica,61070744000177,9.819547,Manufacturing Of Cardboard Packaging And Cardb...,The company Goncalves Industria Grafica was fo...,+55-11-46894700; +55-11-46894728; +55-11-46894...,julio.cesar@goncalves.com.br,Barueri,Sao Paulo,Al Araguaia 1602 Alphaville
521,Josefa Della Volpe,Transportes Della Volpe S A Comercio E Industria,61139432000172,9.817151,Freight Transport.,"Transportes Della Volpe S.A., incorporated in ...",+55-11-29678500; +55-11-29678646; +55-11-29678...,NaN,Sao Paulo (City),Sao Paulo (Region),R Lidice 22 Conj. Promorar Parque Novo Mundo
589,Rafael Della Volpe Filho,Transportes Della Volpe S A Comercio E Industria,61139432000172,9.817151,Freight Transport.,"Transportes Della Volpe S.A., incorporated in ...",+55-11-29678500; +55-11-29678646; +55-11-29678...,NaN,Sao Paulo (City),Sao Paulo (Region),R Lidice 22 Conj. Promorar Parque Novo Mundo


In [33]:
with open('../../Database/EMIS/preprocessed/HWNI_1_shareholder_network_origin_name.csv','w') as myfile:
    for idx, row in df_HNWI_1.iterrows():
        sh_name = row["Shareholder"]
        sh_name_normalized = Normalized_Name_Str(sh_name, patt_non_alphanum)
        if sh_name_normalized in dict_person_id:
            line_temp = []
            line_temp.append(str(idx))
            line_temp.append(re.sub(r',',' ',unidecode(dict_id_person_origin[dict_person_id[sh_name_normalized]])))
            neighs_id_ls = list(dict_id_neighbor[dict_person_id[sh_name_normalized]])
            line_temp.extend([re.sub(r',',' ',unidecode(dict_id_person_origin[neigh_id])) for neigh_id in neighs_id_ls])
            myfile.write(','.join(line_temp)+'\n')
        else:
            print("!!!There is no {} in the dict.".format(row["Shareholder"],))

In [6]:
dict_id_person

{0: 'l s',
 1: 'carmignac gestion',
 2: 'adr jp morgan s',
 3: 'a bunge cooperatier u',
 4: 'antonio dos marcos molina santos',
 5: 'aparecida dos marcia pascoal santos',
 6: 'de isabelle lamoignon marie rose segur',
 7: 'antonio claude de larragoiti lucas patrick',
 8: 'antoinette de marie segur sophie',
 9: 'chantal chevrot de larragoiti lucas',
 10: 'christiane claude de larragoiti lucas',
 11: 'marinho roberto',
 12: 'price rowe t',
 13: 'alfredo arruda egydio filho villela',
 14: 'ana barretto de lucia mattos villela',
 15: 'carlos dias oliveira pires',
 16: 'antonio carlos pipponzi',
 17: 'david feffer',
 18: 'daniel feffer',
 19: 'feffer jorge',
 20: 'feffer ruben',
 21: 'guper janet',
 22: 'lisabeth s sander',
 23: 'pessoas vinculadas',
 24: 'agro industrie tereos',
 25: 'edegel saa',
 26: 'cofal comp franciere',
 27: 'da eggon joao silva',
 28: 'geraldo werninghaus',
 29: 'ricardo voigt werner',
 30: 'david gary neeleman',
 31: 'antonio cunha da luiz seabra',
 32: 'norma pinot

In [7]:
dict_id_comp_info

defaultdict(list,
            {0: [{u'Burberry Brasil Comercio De Artigos De Vestuario E Acessorios Ltda.',
               u'Everis Brasil Consultoria De Negocios E Tecnologia Da Informacao Ltda.',
               u'Ge Digital Energy Do Brasil Ltda',
               u'Irizar Brasil Ltda.',
               u'Kautex Textron Do Brasil Ltda.',
               u'Panda Security Brasil Software E Servicos De Informatica Ltda.',
               u'Prosegur Tecnologia Em Sistemas De Seguranca Eletronica E Incendios Ltda',
               u'S&P Brasil Ventilacao Ltda.',
               u'SP Telecomunicacoes Participacoes Ltda.',
               u'Sanchez Cano Ltda.',
               u'Tecnowatt Iluminacao Ltda.'},
              {u'00891935000105',
               u'01900954000113',
               u'02301582000171',
               u'02320006000171',
               u'03594123000196',
               u'04232671000139',
               u'07616290000141',
               u'10770518000168',
               u'1167445

In [8]:
df_comp_info[df_comp_info['CNPJ']=='16404287000155']

,Index_Num,Company,Key_Executives,Employee_Number,Employee_Info_Year,Financial_Auditors,Main_Products,Phone,Email,Business_Description,...,Status,CNPJ,Audited,Consolidated,Shareholder_Num,Shareholder_Index,Shareholder,Equity,Shareholder_Name_Normalized,Shareholder_Is_Human
254,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,0,Suzano Holding S.A. (Ex-Nemofeffer S.A.),33.72,holding s.a. suzano,False
255,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,1,Bndes Participacoes S.A. - Bndespar,6.86,bndes bndespar participacoes s.a.,False
256,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,2,Mondrian Investment Partners Limited,6.59,investment limited mondrian partners,False
257,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,3,Pessoas Vinculadas,5.20,pessoas vinculadas,True
258,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,4,David Feffer,4.77,david feffer,True
259,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,5,Daniel Feffer,4.35,daniel feffer,True
260,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,6,Ruben Feffer,4.24,feffer ruben,True
261,96.0,Suzano Papel E Celulose S.A.,David Feffer (Chairman of the Board of Directo...,7631.0,2016.0,KPMG,Pulp and paper,NaN,ri@suzano.com.br,"Company headquarted in São Paulo, began operat...",...,0,16404287000155,1,1,8,7,Jorge Feffer,4.20,feffer jorge,True


In [9]:
df_comp_info[df_comp_info['CNPJ']=='60651809000105']

,Index_Num,Company,Key_Executives,Employee_Number,Employee_Info_Year,Financial_Auditors,Main_Products,Phone,Email,Business_Description,...,Status,CNPJ,Audited,Consolidated,Shareholder_Num,Shareholder_Index,Shareholder,Equity,Shareholder_Name_Normalized,Shareholder_Is_Human
248,95.0,Suzano Holding S.A. (Ex-Nemofeffer S.A.),Boris Tabacof (Chairman of the Board of Direct...,34.0,2016.0,KPMG,NaN,+55 11 3503-9320,odias@suzano.com.br,Suzano Holding manages the companies that form...,...,0,60651809000105,1,1,6,0,David Feffer,18.02,david feffer,True
249,95.0,Suzano Holding S.A. (Ex-Nemofeffer S.A.),Boris Tabacof (Chairman of the Board of Direct...,34.0,2016.0,KPMG,NaN,+55 11 3503-9320,odias@suzano.com.br,Suzano Holding manages the companies that form...,...,0,60651809000105,1,1,6,1,Daniel Feffer,17.98,daniel feffer,True
250,95.0,Suzano Holding S.A. (Ex-Nemofeffer S.A.),Boris Tabacof (Chairman of the Board of Direct...,34.0,2016.0,KPMG,NaN,+55 11 3503-9320,odias@suzano.com.br,Suzano Holding manages the companies that form...,...,0,60651809000105,1,1,6,2,Jorge Feffer,17.93,feffer jorge,True
251,95.0,Suzano Holding S.A. (Ex-Nemofeffer S.A.),Boris Tabacof (Chairman of the Board of Direct...,34.0,2016.0,KPMG,NaN,+55 11 3503-9320,odias@suzano.com.br,Suzano Holding manages the companies that form...,...,0,60651809000105,1,1,6,3,Ruben Feffer,17.87,feffer ruben,True
252,95.0,Suzano Holding S.A. (Ex-Nemofeffer S.A.),Boris Tabacof (Chairman of the Board of Direct...,34.0,2016.0,KPMG,NaN,+55 11 3503-9320,odias@suzano.com.br,Suzano Holding manages the companies that form...,...,0,60651809000105,1,1,6,4,Janet Guper,7.01,guper janet,True
253,95.0,Suzano Holding S.A. (Ex-Nemofeffer S.A.),Boris Tabacof (Chairman of the Board of Direct...,34.0,2016.0,KPMG,NaN,+55 11 3503-9320,odias@suzano.com.br,Suzano Holding manages the companies that form...,...,0,60651809000105,1,1,6,5,Lisabeth S. Sander,6.05,lisabeth s sander,True


In [10]:
import re
from unidecode import unidecode

p_cnt = len(dict_person_id)
for idx, row in df_comp_info.iterrows():
    #print("The idx: %s; The names: %s\n" % (idx, row['Key_Executives']))
    try:
        key_execu_name_ls = re.findall(r'([^,]*)\([^()]*\)', row['Key_Executives'])
    except TypeError as e: # The row['Key_Executives'] is float nan.
        print('The key executive names is %s (index: %s).\n' % (row['Key_Executives'], idx))
        key_execu_name_ls = []
    if key_execu_name_ls:
        key_execu_name_normalized_ls = []
        for execu_name in key_execu_name_ls:
            execu_name = unidecode(execu_name.strip().lower())
            #if idx == 636:
            #    print("%s,%s\n" % (idx, execu_name))
            str_ls = execu_name.split(' ')
            str_ls.sort()
            execu_name_normalized = ' '.join(str_ls)
            key_execu_name_normalized_ls.append(execu_name_normalized)
            if not dict_person_id.has_key(execu_name_normalized):
                dict_person_id[execu_name_normalized] = p_cnt
                dict_id_person[p_cnt] = execu_name_normalized
                dict_id_neighbor[p_cnt] = set()
                dict_id_comp_info[p_cnt] = [set(), set()]
                dict_id_comp_info[p_cnt][0].add(row['Company'])
                dict_id_comp_info[p_cnt][1].add(row['CNPJ'])
                p_cnt += 1
        #if idx == 636:
        #    print("%s,%s,%s\n" % (idx, key_execu_name_normalized_ls, row['Shareholder_Name_Normalized']))
        for name in key_execu_name_normalized_ls:
            if row['Shareholder_Is_Human']:
                dict_id_neighbor[dict_person_id[row['Shareholder_Name_Normalized']]].add(dict_person_id[name])
                dict_id_neighbor[dict_person_id[name]].add(dict_person_id[row['Shareholder_Name_Normalized']])
            for neighbor in key_execu_name_normalized_ls:
                if name != neighbor:
                    dict_id_neighbor[dict_person_id[name]].add(dict_person_id[neighbor])
        

The key executive names is nan (index: 197).

The key executive names is nan (index: 436).

The key executive names is nan (index: 474).

The key executive names is nan (index: 511).

The key executive names is nan (index: 528).

The key executive names is nan (index: 574).

The key executive names is nan (index: 633).

The key executive names is nan (index: 662).

The key executive names is nan (index: 680).

The key executive names is nan (index: 721).

The key executive names is nan (index: 745).

The key executive names is nan (index: 1018).

The key executive names is nan (index: 1049).

The key executive names is nan (index: 1139).

The key executive names is nan (index: 1152).

The key executive names is nan (index: 1162).

The key executive names is nan (index: 1166).

The key executive names is nan (index: 1300).

The key executive names is nan (index: 1384).

The key executive names is nan (index: 1397).

The key executive names is nan (index: 1426).

The key executive names 

In [11]:
dict_id_neighbor

defaultdict(set,
            {0: {495,
              1225,
              1661,
              1662,
              1812,
              1842,
              2427,
              3095,
              3096,
              3097,
              5115,
              5116,
              5117,
              6173,
              9059,
              9060,
              9061,
              11445,
              11446,
              11447,
              11448,
              14803,
              14804,
              14805,
              14806,
              14807,
              15607,
              16817,
              18520,
              18521,
              19995,
              19996,
              19997,
              19998,
              19999,
              21188,
              21486,
              21487,
              21488,
              21489,
              21490,
              28107},
             1: {3098,
              3099,
              3100,
              3101,
              3102,
            

In [13]:
with open('person_network.csv','w') as myfile:
    for node, neighs in dict_id_neighbor.iteritems():
        line_temp = []
        line_temp.append(str(node))
        neighs_ls = list(neighs)
        line_temp.extend([str(neigh) for neigh in neighs_ls])
        myfile.write(','.join(line_temp)+'\n')

In [12]:
list(set([1,2,3]))

[1, 2, 3]

### Visualize the network

In [140]:
len(dict_id_neighbor)

31582

In [99]:
re.findall(r'([^,]*)\([^()]*\)', 'Marcelo Jose Teixeira Izzo (CEO), Rodrigo Mattos Assuncao (Executive Vice Presidente), Leonardo Correa (Vice President), Cleber Rocha Vieira (Investor Relation, CFO and CAO), Fabio Goes Acerbi (Investor Relation, CFO and CAO), Rosi Purceti (Director of Human Resources), Bernardo Legey (Sales Director), Bernardo Jereissati Legey (Director), Luiz Carlos Rodrigues (Director), Nilson Roberto Tagliari (Director), Mario Antonio Veronezzi (Supplies Director)')

['Marcelo Jose Teixeira Izzo ',
 ' Rodrigo Mattos Assuncao ',
 ' Leonardo Correa ',
 ' Cleber Rocha Vieira ',
 ' Fabio Goes Acerbi ',
 ' Rosi Purceti ',
 ' Bernardo Legey ',
 ' Bernardo Jereissati Legey ',
 ' Luiz Carlos Rodrigues ',
 ' Nilson Roberto Tagliari ',
 ' Mario Antonio Veronezzi ']

In [68]:
dict_person_id['breno de oliveira pacheco rodrigo']

KeyError: 'breno de oliveira pacheco rodrigo'

In [65]:
str_ls.sort()

In [122]:
df_comp_info[df_comp_info['CNPJ']=='01900954000113']

,Index_Num,Company,Key_Executives,Employee_Number,Employee_Info_Year,Financial_Auditors,Main_Products,Phone,Email,Business_Description,...,Status,CNPJ,Audited,Consolidated,Shareholder_Num,Shareholder_Index,Shareholder,Equity,Shareholder_Name_Normalized,Shareholder_Is_Human
46,17.0,SP Telecomunicacoes Participacoes Ltda.,"Breno Rodrigo Pacheco de Oliveira (Director), ...",NaN,NaN,EY,NaN,+55 11 3047-1733 / +55 11 3430-0000,telefonica@telefonica.com,The company Sp Telecomunicacoes Participacoes ...,...,0,01900954000113,1,1,3,0,Telefonica Latinoamerica Holding,NaN,holding latinoamerica telefonica,False
47,17.0,SP Telecomunicacoes Participacoes Ltda.,"Breno Rodrigo Pacheco de Oliveira (Director), ...",NaN,NaN,EY,NaN,+55 11 3047-1733 / +55 11 3430-0000,telefonica@telefonica.com,The company Sp Telecomunicacoes Participacoes ...,...,0,01900954000113,1,1,3,1,S.L.,NaN,l s,True
48,17.0,SP Telecomunicacoes Participacoes Ltda.,"Breno Rodrigo Pacheco de Oliveira (Director), ...",NaN,NaN,EY,NaN,+55 11 3047-1733 / +55 11 3430-0000,telefonica@telefonica.com,The company Sp Telecomunicacoes Participacoes ...,...,0,01900954000113,1,1,3,2,Telefonica S.A.,NaN,s.a. telefonica,False


In [121]:
for idx,row in temp.iterrows():
    print(row)

Index_Num                                                                     17
Company                                  SP Telecomunicacoes Participacoes Ltda.
Key_Executives                 Breno Rodrigo Pacheco de Oliveira (Director), ...
Employee_Number                                                              NaN
Employee_Info_Year                                                           NaN
Financial_Auditors                                                            EY
Main_Products                                                                NaN
Phone                                        +55 11 3047-1733 / +55 11 3430-0000
Email                                                  telefonica@telefonica.com
Business_Description           The company Sp Telecomunicacoes Participacoes ...
Shareholder_String             Telefonica Latinoamerica Holding, S.L., Telefo...
Revenue_2017                                                                 NaN
Revenue_2016                

In [42]:
df_comp_info[197:198]['Key_Executives'].item()

nan

In [45]:
df_comp_info[196:197]['Key_Executives'].item()

u'Alfredo Egydio Arruda Villela Filho (Chairman of the Board of Directors), Henri Penchas (Member of Board of Directors), Licio da Costa Raimundo (Member of Board of Directors), Paulo Setubal Neto (Member of Board of Directors), Ricardo Egydio Setubal (Member of Board of Directors), Ricardo Villela Marino (Member of Board of Directors), Rodolfo Villela Marino (Member of Board of Directors and Vice President), Alexandre Barenco Ribeiro (Board of Directors - Council), Augusto Carneiro de Oliveira Filho (Board of Directors - Council), Felicio Cintra do Prado Junior (Board of Directors - Council), Flavio Cesar Maia Luz (Board of Directors - Council), Joao Costa (Board of Directors - Council), Jose Carlos de Brito e Cunha (Board of Directors - Council), Jose Roberto Brant de Carvalho (Board of Directors - Council), Paulo Ricardo Moraes Amaral (Board of Directors - Council), Pedro Americo Herbst (Board of Directors - Council), Tereza Cristina Grossi Togni (Board of Directors - Council), Alfr